In [1]:
import pandas as pd
import os
os.chdir(r"C:\Users\7LStar\Desktop\study\python\同期群分析")
df=pd.read_excel('同期群分析数据.xlsx')

In [2]:
df.head()

,品牌名,店铺名称,主订单编号,用户ID,付款时间,实付金额
0,阿粥（小z）,数据不吹牛,73465136654,uid135460366,2023-01-01 09:32:12,166
1,阿粥（小z）,数据不吹牛,73465136655,uid135460367,2023-01-01 09:11:50,117
2,阿粥（小z）,数据不吹牛,73465136656,uid135460368,2023-01-01 11:49:02,166
3,阿粥（小z）,数据不吹牛,73465136657,uid135460369,2023-01-01 12:20:24,77
4,阿粥（小z）,数据不吹牛,73465136658,uid135460370,2023-01-01 01:23:15,158


In [3]:
df['时间标签']=df['付款时间'].astype(str).str[:7]
df['时间标签'].value_counts().sort_index()

2023-01    12039
2023-02     4114
2023-03    18172
2023-04    12320
2023-05    15738
2023-06    44265
2023-07    16102
2023-08    28817
2023-09    60376
2023-10    21639
2023-11    40204
2023-12    15600
Name: 时间标签, dtype: int64

In [4]:
month_lst=df['时间标签'].unique()
final=pd.DataFrame()

In [5]:
for i in range(len(month_lst) - 1):
    
    #构造和月份一样长的列表，方便后续格式统一
    count = [0] * len(month_lst)
    
    #筛选出当月订单，并按用户昵称分组
    target_month = df.loc[df['时间标签'] == month_lst[i],:]
    target_users = target_month.groupby('用户ID')['实付金额'].sum().reset_index()
    
    #如果是第一个月，则跳过（因为不需要和历史数据验证是否为新增用户）
    if i == 0:
        new_target_users = target_month.groupby('用户ID')['实付金额'].sum().reset_index()
    else:
        #如果不是，找到之前的历史订单
        history = df.loc[df['时间标签'].isin(month_lst[:i]),:]
        #筛选出未在历史订单中出现过的新增用户
        new_target_users = target_users.loc[target_users['用户ID'].isin(history['用户ID']) == False,:]
    
    #将当月新增用户数放在第一个值中
    count[0] = len(new_target_users)
    
    #以月为单位，循环遍历，计算留存情况
    for j,ct in zip(range(i + 1,len(month_lst)),range(1,len(month_lst))):
        #下一个月的订单
        next_month = df.loc[df['时间标签'] == month_lst[j],:]
        next_users = next_month.groupby('用户ID')['实付金额'].sum().reset_index()
        #计算在该月仍然留存的用户数量
        isin = new_target_users['用户ID'].isin(next_users['用户ID']).sum()
        count[ct] = isin
        
    #格式转置
    result = pd.DataFrame({month_lst[i]:count}).T
    
    #合并
    final = pd.concat([final,result])

final.columns = ['当月新增','+1月','+2月','+3月','+4月','+5月','+6月','+7月','+8月','+9月','+10月','+11月']

final

,当月新增,+1月,+2月,+3月,+4月,+5月,+6月,+7月,+8月,+9月,+10月,+11月
2023-01,8193,573,1601,1050,1079,1906,815,1102,863,628,1049,372
2023-02,2740,558,340,379,587,293,317,267,205,304,112,0
2023-03,8753,1176,1232,2112,799,1032,777,616,1064,360,0,0
2023-04,5859,828,1208,502,618,482,329,526,171,0,0,0
2023-05,6912,1575,626,747,464,392,569,198,0,0,0,0
2023-06,16458,1575,1775,1153,923,1482,496,0,0,0,0,0
2023-07,6514,801,404,257,390,144,0,0,0,0,0,0
2023-08,11781,1030,606,813,331,0,0,0,0,0,0,0
2023-09,30214,2206,2482,971,0,0,0,0,0,0,0,0
2023-10,11253,1147,452,0,0,0,0,0,0,0,0,0


In [6]:
#引入时间标签
month_lst = df['时间标签'].unique()
final = pd.DataFrame()

for i in range(len(month_lst) - 1):
    
    #构造和月份一样长的列表，方便后续格式统一
    count = [0] * len(month_lst)
    
    #筛选出当月订单，并按用户昵称分组
    target_month = df.loc[df['时间标签'] == month_lst[i],:]
    target_users = target_month.groupby('用户ID')['实付金额'].sum().reset_index()
    
    #如果是第一个月，则跳过（因为不需要和历史数据验证是否为新增用户）
    if i == 0:
        new_target_users = target_month.groupby('用户ID')['实付金额'].sum().reset_index()
    else:
        #如果不是，找到之前的历史订单
        history = df.loc[df['时间标签'].isin(month_lst[:i]),:]
        #筛选出未在历史订单中出现过的新增用户
        new_target_users = target_users.loc[target_users['用户ID'].isin(history['用户ID']) == False,:]
    
    #将当月新增用户数放在第一个值中
    count[0] = len(new_target_users)
    
    #以月为单位，循环遍历，计算留存情况
    for j,ct in zip(range(i + 1,len(month_lst)),range(1,len(month_lst))):
        #下一个月的订单
        next_month = df.loc[df['时间标签'] == month_lst[j],:]
        next_users = next_month.groupby('用户ID')['实付金额'].sum().reset_index()
        #计算在该月仍然留存的用户数量
        isin = new_target_users['用户ID'].isin(next_users['用户ID']).sum()
        count[ct] = isin
        
    #格式转置
    result = pd.DataFrame({month_lst[i]:count}).T
    
    #合并
    final = pd.concat([final,result])

final.columns = ['当月新增','+1月','+2月','+3月','+4月','+5月','+6月','+7月','+8月','+9月','+10月','+11月']

final

,当月新增,+1月,+2月,+3月,+4月,+5月,+6月,+7月,+8月,+9月,+10月,+11月
2023-01,8193,573,1601,1050,1079,1906,815,1102,863,628,1049,372
2023-02,2740,558,340,379,587,293,317,267,205,304,112,0
2023-03,8753,1176,1232,2112,799,1032,777,616,1064,360,0,0
2023-04,5859,828,1208,502,618,482,329,526,171,0,0,0
2023-05,6912,1575,626,747,464,392,569,198,0,0,0,0
2023-06,16458,1575,1775,1153,923,1482,496,0,0,0,0,0
2023-07,6514,801,404,257,390,144,0,0,0,0,0,0
2023-08,11781,1030,606,813,331,0,0,0,0,0,0,0
2023-09,30214,2206,2482,971,0,0,0,0,0,0,0,0
2023-10,11253,1147,452,0,0,0,0,0,0,0,0,0


In [9]:
result=final.divide(final['当月新增'],axis=0).iloc[:,1:]
result['当月新增']=final['当月新增']
result

,+1月,+2月,+3月,+4月,+5月,+6月,+7月,+8月,+9月,+10月,+11月,当月新增
2023-01,0.069938,0.195411,0.128158,0.131698,0.232638,0.099475,0.134505,0.105334,0.076651,0.128036,0.045405,8193
2023-02,0.203650,0.124088,0.138321,0.214234,0.106934,0.115693,0.097445,0.074818,0.110949,0.040876,0.000000,2740
2023-03,0.134354,0.140752,0.241289,0.091283,0.117902,0.088770,0.070376,0.121558,0.041129,0.000000,0.000000,8753
2023-04,0.141321,0.206179,0.085680,0.105479,0.082267,0.056153,0.089776,0.029186,0.000000,0.000000,0.000000,5859
2023-05,0.227865,0.090567,0.108073,0.067130,0.056713,0.082321,0.028646,0.000000,0.000000,0.000000,0.000000,6912
2023-06,0.095698,0.107850,0.070057,0.056082,0.090047,0.030137,0.000000,0.000000,0.000000,0.000000,0.000000,16458
2023-07,0.122966,0.062020,0.039453,0.059871,0.022106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6514
2023-08,0.087429,0.051439,0.069009,0.028096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11781
2023-09,0.073013,0.082147,0.032137,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30214
2023-10,0.101928,0.040167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11253
